## Quick look at the submition for the verification binaries 

In [ ]:
### importing modules

import numpy as np
import os, sys, re
import time
import glob

from LISAhdf5 import LISAhdf5,ParsUnits
import tdi

import yaml, pandas, csv

import FastGB

# plotting
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl


import corner

#### useful functions and parameters 


In [ ]:
### Some settings
plotpal = ["#4C72B0", "#C44E52", "#CCB974", "#55A868", "#8172B2", "#64B5CD"]

### Get parameters of the source specified by index
def GetParams(p, ind):
    bet = p.get('EclipticLatitude')[ind]
    lam = p.get('EclipticLongitude')[ind]
    Amp = p.get("Amplitude")[ind]
    f0 = p.get("Frequency")[ind]
    fdot = p.get("FrequencyDerivative")[ind]
    iota = p.get("Inclination")[ind]
    psi = p.get("Polarization")[ind]
    phi0 = p.get("InitialPhase")[ind]

    return (bet, lam, Amp, f0, fdot, iota, psi, phi0)


### Window function used in fft
def Window(tm, offs=1000.0):

    xl = offs
    ind_r = np.argwhere(tm[-1]-tm <= offs)[0][0]
    xr = tm[ind_r]
    # print (xr)
    kap = 0.005
    winl = 0.5*(1.0 + np.tanh(kap*(tm-xl)))
    winr = 0.5*(1.0 - np.tanh(kap*(tm-xr)))
    # plt.plot(tm, winl)
    # plt.plot(tm, winr)
    # plt.grid(True)
    # plt.show()
    return (winl*winr)


from scipy import stats
def jensen_shannon_divergence(
    samples, kde=stats.gaussian_kde, decimal=5, base=np.e, **kwargs
):
    """Calculate the JS divergence between two sets of samples

    Parameters
    ----------
    samples: list
        2d list containing the samples drawn from two pdfs
    kde: func
        function to use when calculating the kde of the samples
    decimal: int, float
        number of decimal places to round the JS divergence to
    base: float, optional
        optional base to use for the scipy.stats.entropy function. Default
        np.e
    kwargs: dict
        all kwargs are passed to the kde function
    """
    try:
        kernel = [kde(i, **kwargs) for i in samples]
    except np.linalg.LinAlgError:
        return float("nan")
    x = np.linspace(
        np.min([np.min(i) for i in samples]),
        np.max([np.max(i) for i in samples]),
        100
    )
    a, b = [k(x) for k in kernel]
    a = np.asarray(a)
    b = np.asarray(b)
    a /= a.sum()
    b /= b.sum()
    m = 1. / 2 * (a + b)
    kl_forward = stats.entropy(a, qk=m, base=base)
    kl_backward = stats.entropy(b, qk=m, base=base)
    return np.round(kl_forward / 2. + kl_backward / 2., decimal)


### Reading the data for VGBs and performing FFT

In [ ]:



DATAPATH = "/home/stefan/LDC/Radler/data"
sangria_fn = DATAPATH + "/LDC1-3_VGB_v2.hdf5"
FD5 = LISAhdf5(sangria_fn)
Nsrc = FD5.getSourcesNum()
GWs = FD5.getSourcesName()
print ("Found %d GW sources: " % Nsrc, GWs)
### TODO make sure GalBin is there
if (GWs[0] != 'GalBinaries'):
    raise NotImplementedError
p = FD5.getSourceParameters(GWs[0])
td = FD5.getPreProcessTDI()
del_t = float(p.get("Cadence"))
Tobs = float(p.get("ObservationDuration"))

p.display()
Nsrc = len(p.get('Frequency'))

df = 1.0/Tobs
XDt = td[:, 1]
tm = td[:, 0]
win = Window(tm)


XDf = np.fft.fft(win*td[:, 1])
YDf = np.fft.fft(win*td[:, 2])
ZDf = np.fft.fft(win*td[:, 3])
freqD = np.fft.fftfreq(len(tm), del_t)
 
Nfd = len(freqD)
Nfd = int(Nfd/2)
freqD = freqD[:Nfd]  ### we use only positive frequencies
Adf = (ZDf[:Nfd] - XDf[:Nfd])/np.sqrt(2.0)
Edf = (ZDf[:Nfd] - 2.0*YDf[:Nfd] + XDf[:Nfd])/np.sqrt(6.0)
Adf = Adf * del_t
Edf = Edf * del_t  ### Due to fft conventions


fastGB = FastGB.FastGB("Test", dt=del_t, Tobs=Tobs, orbit="analytic")

In [ ]:
### Dictionary of events: need to map different submission index to VGBs in the catalogue

VGBs = {'GW135962': {'BC':3, 'MM':1, 'BH':6, 'S':0},  
        'GW125313': {'BC':4, 'MM':7, 'BH':4, 'S':1},
        'GW181324': {'BC':8, 'MM':2, 'BH':5, 'S':2},
        'GW166667': {'BC':1, 'MM':9, 'BH':3, 'S':3},
        'GW194414': {'BC':5, 'MM':0, 'BH':9, 'S':4},
        'GW322061': {'BC':6, 'MM':5, 'BH':1, 'S':5},
        'GW351250': {'BC':0, 'MM':6, 'BH':2, 'S':6}, 
        'GW168350': {'BC':2, 'MM':3, 'BH':8, 'S':7},
        'GW622028': {'BC':7, 'MM':8, 'BH':7, 'S':8},
        'GW261301': {'BC':9, 'MM':4, 'BH':0, 'S':9}}

### Read Barcelona data (from ice-csic-ieec)

In [ ]:
Barc_data = []
Barc_data_pnd = []

fls = glob.glob('/home/stefan/Repositories/ldc1_evaluation_data/submission/Barcelona/ldc1-3_posteriors_ice-csic-ieec_v3/*.csv')
print (fls)

# with open(fls[0]) as csv_file:
#     csv_reader = csv.reader(csv_file, delimiter=',')
#     for raw in csv_reader:
#         print (raw)
#         break

for fl in fls:
#     df = pandas.read_csv(fl)
#     Barc_data_pnd.append(df)
    dat = np.genfromtxt(fl, delimiter=',', names=True)[::100]
    Barc_data.append(dat)
    
print (len(dat['EclipticLongitude']))
    

### Reading posterior from B-gham 

In [ ]:
### posteriors from B-gham
Bham_data = []
Bham_data_pnd = []
fls = glob.glob('/home/stefan/Repositories/ldc1_evaluation_data/submission/BirminghamU/buscicchio-ldc1-3-vgb_posteriors_csv/vgb*.csv')
print (fls)

# with open(fls[0]) as csv_file:
#     csv_reader = csv.reader(csv_file, delimiter=',')
#     for raw in csv_reader:
#         print (raw)
#         break

for fl in fls:
    df = pandas.read_csv(fl)
    Bham_data_pnd.append(df)
    dat = np.genfromtxt(fl, delimiter=',', names=True)
    Bham_data.append(dat)
    
print (np.shape(dat))
print (len(dat['EclipticLongitude']))

### Reading posterior from Marshall-Montana

In [ ]:
### posteriors from Nasa
MM_data = []
MM_data_pnd = []
fls = glob.glob('/home/stefan/Repositories/ldc1_evaluation_data/submission/MarshallMontana/vgb-3-pdf/*.csv')
print (fls)

# with open(fls[0]) as csv_file:
#     csv_reader = csv.reader(csv_file, delimiter=',')
#     for raw in csv_reader:
#         print (raw)
#         break

for fl in fls:
    df = pandas.read_csv(fl)
    MM_data_pnd.append(df)
    dat = np.genfromtxt(fl, delimiter=',', names=True)
    MM_data.append(dat)
    
print (np.shape(dat))
print (len(dat['EclipticLongitude']))

### Reading posterior from Stefan

In [ ]:
### posteriors from Strub
S_data = []
S_data_pnd = []
for name,dict_ in VGBs.items():
    try:
        fl = glob.glob('/home/stefan/Repositories/ldc1_evaluation_data/submission/ETH_LDC1-3-VGB_reduced_20k/'+str(name)+'.csv')[0]
        print('fl',fl)
        df = pandas.read_csv(fl)
        S_data_pnd.append(df)
        dat = np.genfromtxt(fl, delimiter=',', names=True)
        S_data.append(dat)
    except:
        pass
    
print (np.shape(dat))
print (len(dat['EclipticLongitude']))

In [ ]:
from matplotlib import colors as mcolors
lbls = [r'$\lambda$', r'$\beta$', 'f(Hz)', r'$\dot{f}$', r'$\iota$', r'$A$']

ind_0 = 0
name = 'GW135962'
inds_d = VGBs[name]

ind = VGBs[name]['BH']
datBh = np.zeros((len(Bham_data[ind]), len(lbls)))
datBh[:, 0] = Bham_data[ind]['EclipticLongitude'] -np.pi
datBh[:, 1] = Bham_data[ind]['EclipticLatitude']
datBh[:, 2] = Bham_data[ind]['Frequency']
datBh[:, 3] = np.log10(np.abs(Bham_data[ind]['FrequencyDerivative']))
datBh[:, 4] = Bham_data[ind]['Inclination']
datBh[:, 5] = np.log10(Bham_data[ind]['Amplitude'])

print (datBh[:, 3])

ind = VGBs[name]['MM']
datM = np.zeros((len(MM_data[ind]), len(lbls)))
datM[:, 0] = MM_data[ind]['EclipticLongitude']-2.0*np.pi
datM[:, 1] = MM_data[ind]['EclipticLatitude']
datM[:, 2] = MM_data[ind]['Frequency']
datM[:, 3] = np.log10(np.abs(MM_data[ind]['FrequencyDerivative']))
datM[:, 4] = MM_data[ind]['Inclination']
datM[:, 5] = np.log10(MM_data[ind]['Amplitude'])

ind = VGBs[name]['BC']
datBc = np.zeros((len(Barc_data[ind]), len(lbls)))
datBc[:, 0] = Barc_data[ind]['EclipticLongitude']
datBc[:, 1] = Barc_data[ind]['EclipticLatitude']
datBc[:, 2] = Barc_data[ind]['Frequency']
datBc[:, 3] = np.log10(np.abs(Barc_data[ind]['FrequencyDerivative']))
datBc[:, 4] = np.arccos(np.cos(Barc_data[ind]['Inclination']))
datBc[:, 5] = np.log10(Barc_data[ind]['Amplitude'])

ind = VGBs[name]['S']
print(ind)
datS = np.zeros((len(S_data[ind]), len(lbls)))
datS[:, 0] = S_data[ind]['EclipticLongitude']
datS[:, 1] = S_data[ind]['EclipticLatitude']
datS[:, 2] = S_data[ind]['Frequency']
datS[:, 3] = np.log10(np.abs(S_data[ind]['FrequencyDerivative']))
datS[:, 4] = np.arccos(np.cos(S_data[ind]['Inclination']))
datS[:, 5] = np.log10(S_data[ind]['Amplitude'])

bet, lam, Amp, f0, fdot, iota, psi, phi0 = GetParams(p, ind_0)

tr_s = np.array([lam, bet, f0, np.log10(fdot), iota, np.log10(Amp)])
print ("true params", tr_s)
print ('M', datM[:3, 0])
print ('Bh', datBh[:3,0])
print ('Bc', datBc[:3,0])
print ('S', datS[:3,0])

rng = [0.999, 0.999, 0.999, (-20.5,-15.5), 0.99, 0.99]
rng = []
# print (datBh.min(), datBc.min(), datM.min())
# print (max(datBh.max(), datBc.max(), datM.max()))
for i in range(len(lbls)):
    oner = ( min(datBh[:,i].min(), datS[:, i].min(), datM[:,i].min()), max(datBh[:, i].max(), datS[:,i].max(), datM[:,i].max()) )
    rng.append(oner)

fig2 =  corner.corner(datBh,  bins=40, hist_kwargs={'density':True, 'lw':3}, plot_datapoints=False, fill_contours=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='#348ABD', truths= tr_s, truth_color='k', use_math_test=True,\
                        levels=[0.9], title_kwargs={"fontsize": 12})

# corner.corner(datBc,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
#                         color=plotpal[3], truths= tr_s, truth_color='k', use_math_test=True,\
#                          levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datS,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color=plotpal[3], truths= tr_s, truth_color='k', use_math_test=True,\
                         levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datM,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, fill_contours=False,  labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='coral', truths= tr_s, truth_color='k', use_math_test=True,\
                           range=rng, levels=[0.9], title_kwargs={"fontsize": 12})



plt.show()
fig2.savefig("/home/stefan/Repositories/ldc1_evaluation/Stefan/VGB_"+str(ind_0)+".png")

# fig3 = corner.corner(datM,  bins=40,  hist_kwargs={'density':True},  labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
#                         color='#348ABD', truths= tr_s, truth_color='k', use_math_test=True,\
#                         range=rng, title_kwargs={"fontsize": 12})


In [ ]:
from matplotlib import colors as mcolors
lbls = [r'$\lambda$', r'$\beta$', 'f(Hz)', r'$\dot{f}$', r'$\iota$', r'$A$']

ind_0 = 1
name = 'GW125313'
inds_d = VGBs[name]

ind = VGBs[name]['BH']
datBh = np.zeros((len(Bham_data[ind]), len(lbls)))
datBh[:, 0] = Bham_data[ind]['EclipticLongitude'] -np.pi
datBh[:, 1] = Bham_data[ind]['EclipticLatitude']
datBh[:, 2] = Bham_data[ind]['Frequency']
datBh[:, 3] = np.log10(np.abs(Bham_data[ind]['FrequencyDerivative']))
datBh[:, 4] = Bham_data[ind]['Inclination']
datBh[:, 5] = np.log10(Bham_data[ind]['Amplitude'])

print (datBh[:, 3])

ind = VGBs[name]['MM']
datM = np.zeros((len(MM_data[ind]), len(lbls)))
datM[:, 0] = MM_data[ind]['EclipticLongitude']-2.0*np.pi
datM[:, 1] = MM_data[ind]['EclipticLatitude']
datM[:, 2] = MM_data[ind]['Frequency']
datM[:, 3] = np.log10(np.abs(MM_data[ind]['FrequencyDerivative']))
datM[:, 4] = MM_data[ind]['Inclination']
datM[:, 5] = np.log10(MM_data[ind]['Amplitude'])

ind = VGBs[name]['BC']
datBc = np.zeros((len(Barc_data[ind]), len(lbls)))
datBc[:, 0] = Barc_data[ind]['EclipticLongitude']
datBc[:, 1] = Barc_data[ind]['EclipticLatitude']
datBc[:, 2] = Barc_data[ind]['Frequency']
datBc[:, 3] = np.log10(np.abs(Barc_data[ind]['FrequencyDerivative']))
datBc[:, 4] = np.arccos(np.cos(Barc_data[ind]['Inclination']))
datBc[:, 5] = np.log10(Barc_data[ind]['Amplitude'])

ind = VGBs[name]['S']
datS = np.zeros((len(S_data[ind]), len(lbls)))
datS[:, 0] = S_data[ind]['EclipticLongitude']
datS[:, 1] = S_data[ind]['EclipticLatitude']
datS[:, 2] = S_data[ind]['Frequency']
datS[:, 3] = np.log10(np.abs(S_data[ind]['FrequencyDerivative']))
datS[:, 4] = np.arccos(np.cos(S_data[ind]['Inclination']))
datS[:, 5] = np.log10(S_data[ind]['Amplitude'])

bet, lam, Amp, f0, fdot, iota, psi, phi0 = GetParams(p, ind_0)

tr_s = np.array([lam, bet, f0, np.log10(fdot), iota, np.log10(Amp)])
print ("true params", tr_s)
print ('M', datM[:3, 0])
print ('Bh', datBh[:3,0])
print ('Bc', datBc[:3,0])

rng = [0.999, 0.999, 0.999, (-20.5,-15.5), 0.99, 0.99]
rng = []
# print (datBh.min(), datBc.min(), datM.min())
# print (max(datBh.max(), datBc.max(), datM.max()))
for i in range(len(lbls)):
    oner = ( min(datBh[:,i].min(), datS[:, i].min(), datM[:,i].min()), max(datBh[:, i].max(), datS[:,i].max(), datM[:,i].max()) )
    rng.append(oner)

fig2 =  corner.corner(datBh,  bins=40, hist_kwargs={'density':True, 'lw':3}, plot_datapoints=False, fill_contours=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='#348ABD', truths= tr_s, truth_color='k', use_math_test=True,\
                        levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datS,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color=plotpal[3], truths= tr_s, truth_color='k', use_math_test=True,\
                         levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datM,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, fill_contours=False,  labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='coral', truths= tr_s, truth_color='k', use_math_test=True,\
                         range=rng,levels=[0.9], title_kwargs={"fontsize": 12})



plt.show()
fig2.savefig("/home/stefan/Repositories/ldc1_evaluation/Stefan/VGB_"+str(ind_0)+".png")

# fig3 = corner.corner(datM,  bins=40,  hist_kwargs={'density':True},  labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
#                         color='#348ABD', truths= tr_s, truth_color='k', use_math_test=True,\
#                         title_kwargs={"fontsize": 12})


In [ ]:
ind_0 = 2
name = 'GW181324'
inds_d = VGBs[name]

ind = VGBs[name]['BH']
datBh = np.zeros((len(Bham_data[ind]), len(lbls)))
datBh[:, 0] = Bham_data[ind]['EclipticLongitude'] -np.pi
datBh[:, 1] = Bham_data[ind]['EclipticLatitude']
datBh[:, 2] = Bham_data[ind]['Frequency']
datBh[:, 3] = np.log10(np.abs(Bham_data[ind]['FrequencyDerivative']))
datBh[:, 4] = Bham_data[ind]['Inclination']
datBh[:, 5] = np.log10(Bham_data[ind]['Amplitude'])

print (datBh[:, 3])

ind = VGBs[name]['MM']
datM = np.zeros((len(MM_data[ind]), len(lbls)))
datM[:, 0] = MM_data[ind]['EclipticLongitude']-2.0*np.pi
datM[:, 1] = MM_data[ind]['EclipticLatitude']
datM[:, 2] = MM_data[ind]['Frequency']
datM[:, 3] = np.log10(np.abs(MM_data[ind]['FrequencyDerivative']))
datM[:, 4] = MM_data[ind]['Inclination']
datM[:, 5] = np.log10(MM_data[ind]['Amplitude'])

ind = VGBs[name]['BC']
datBc = np.zeros((len(Barc_data[ind]), len(lbls)))
datBc[:, 0] = Barc_data[ind]['EclipticLongitude']
datBc[:, 1] = Barc_data[ind]['EclipticLatitude']
datBc[:, 2] = Barc_data[ind]['Frequency']
datBc[:, 3] = np.log10(np.abs(Barc_data[ind]['FrequencyDerivative']))
datBc[:, 4] = np.arccos(np.cos(Barc_data[ind]['Inclination']))
datBc[:, 5] = np.log10(Barc_data[ind]['Amplitude'])

ind = VGBs[name]['S']
datS = np.zeros((len(S_data[ind]), len(lbls)))
datS[:, 0] = S_data[ind]['EclipticLongitude']
datS[:, 1] = S_data[ind]['EclipticLatitude']
datS[:, 2] = S_data[ind]['Frequency']
datS[:, 3] = np.log10(np.abs(S_data[ind]['FrequencyDerivative']))
datS[:, 4] = np.arccos(np.cos(S_data[ind]['Inclination']))
datS[:, 5] = np.log10(S_data[ind]['Amplitude'])

bet, lam, Amp, f0, fdot, iota, psi, phi0 = GetParams(p, ind_0)

tr_s = np.array([lam, bet, f0, np.log10(fdot), iota, np.log10(Amp)])
print ("true params", tr_s)
print ('M', datM[:3, 0])
print ('Bh', datBh[:3,0])
print ('Bc', datBc[:3,0])

rng = [0.999, 0.999, 0.999, (-20.5,-15.5), 0.99, 0.99]
rng = []
# print (datBh.min(), datBc.min(), datM.min())
# print (max(datBh.max(), datBc.max(), datM.max()))
for i in range(len(lbls)):
    oner = ( min(datBh[:,i].min(), datS[:, i].min(), datM[:,i].min()), max(datBh[:, i].max(), datS[:,i].max(), datM[:,i].max()) )
    rng.append(oner)

fig2 =  corner.corner(datBh,  bins=40, hist_kwargs={'density':True, 'lw':3}, plot_datapoints=False, fill_contours=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='#348ABD', truths= tr_s, truth_color='k', use_math_test=True,\
                        levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datS,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color=plotpal[3], truths= tr_s, truth_color='k', use_math_test=True,\
                         levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datM,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, fill_contours=False,  labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='coral', truths= tr_s, truth_color='k', use_math_test=True,\
                         range=rng, levels=[0.9], title_kwargs={"fontsize": 12})



plt.show()
fig2.savefig("/home/stefan/Repositories/ldc1_evaluation/Stefan/VGB_"+str(ind_0)+".png")

In [ ]:
ind_0 = 3
name = 'GW166667'
inds_d = VGBs[name]

ind = VGBs[name]['BH']
datBh = np.zeros((len(Bham_data[ind]), len(lbls)))
datBh[:, 0] = Bham_data[ind]['EclipticLongitude'] -np.pi
datBh[:, 1] = Bham_data[ind]['EclipticLatitude']
datBh[:, 2] = Bham_data[ind]['Frequency']
datBh[:, 3] = np.log10(np.abs(Bham_data[ind]['FrequencyDerivative']))
datBh[:, 4] = Bham_data[ind]['Inclination']
datBh[:, 5] = np.log10(Bham_data[ind]['Amplitude'])

print (datBh[:, 3])

ind = VGBs[name]['MM']
datM = np.zeros((len(MM_data[ind]), len(lbls)))
datM[:, 0] = MM_data[ind]['EclipticLongitude']
datM[:, 1] = MM_data[ind]['EclipticLatitude']
datM[:, 2] = MM_data[ind]['Frequency']
datM[:, 3] = np.log10(np.abs(MM_data[ind]['FrequencyDerivative']))
datM[:, 4] = MM_data[ind]['Inclination']
datM[:, 5] = np.log10(MM_data[ind]['Amplitude'])

ind = VGBs[name]['BC']
datBc = np.zeros((len(Barc_data[ind]), len(lbls)))
datBc[:, 0] = Barc_data[ind]['EclipticLongitude']
datBc[:, 1] = Barc_data[ind]['EclipticLatitude']
datBc[:, 2] = Barc_data[ind]['Frequency']
datBc[:, 3] = np.log10(np.abs(Barc_data[ind]['FrequencyDerivative']))
datBc[:, 4] = np.arccos(np.cos(Barc_data[ind]['Inclination']))
datBc[:, 5] = np.log10(Barc_data[ind]['Amplitude'])

ind = VGBs[name]['S']
datS = np.zeros((len(S_data[ind]), len(lbls)))
datS[:, 0] = S_data[ind]['EclipticLongitude']
datS[:, 1] = S_data[ind]['EclipticLatitude']
datS[:, 2] = S_data[ind]['Frequency']
datS[:, 3] = np.log10(np.abs(S_data[ind]['FrequencyDerivative']))
datS[:, 4] = np.arccos(np.cos(S_data[ind]['Inclination']))
datS[:, 5] = np.log10(S_data[ind]['Amplitude'])

bet, lam, Amp, f0, fdot, iota, psi, phi0 = GetParams(p, ind_0)

tr_s = np.array([lam, bet, f0, np.log10(fdot), iota, np.log10(Amp)])
print ("true params", tr_s)
print ('M', datM[:3, 0])
print ('Bh', datBh[:3,0])
print ('Bc', datBc[:3,0])

rng = [0.999, 0.999, 0.999, (-20.5,-15.5), 0.99, 0.99]
rng = []
# print (datBh.min(), datBc.min(), datM.min())
# print (max(datBh.max(), datBc.max(), datM.max()))
for i in range(len(lbls)):
    oner = ( min(datBh[:,i].min(), datS[:, i].min(), datM[:,i].min()), max(datBh[:, i].max(), datS[:,i].max(), datM[:,i].max()) )
    rng.append(oner)

fig2 =  corner.corner(datBh,  bins=40, hist_kwargs={'density':True, 'lw':3}, plot_datapoints=False, fill_contours=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='#348ABD', truths= tr_s, truth_color='k', use_math_test=True,\
                        levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datS,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color=plotpal[3], truths= tr_s, truth_color='k', use_math_test=True,\
                         levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datM,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, fill_contours=False,  labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='coral', truths= tr_s, truth_color='k', use_math_test=True,\
                         range=rng, levels=[0.9], title_kwargs={"fontsize": 12})

plt.show()
fig2.savefig("/home/stefan/Repositories/ldc1_evaluation/Stefan/VGB_"+str(ind_0)+".png")

In [ ]:
ind_0 = 4
name = 'GW194414'
inds_d = VGBs[name]

ind = VGBs[name]['BH']
datBh = np.zeros((len(Bham_data[ind]), len(lbls)))
datBh[:, 0] = Bham_data[ind]['EclipticLongitude'] -np.pi
datBh[:, 1] = Bham_data[ind]['EclipticLatitude']
datBh[:, 2] = Bham_data[ind]['Frequency']
datBh[:, 3] = np.log10(np.abs(Bham_data[ind]['FrequencyDerivative']))
datBh[:, 4] = Bham_data[ind]['Inclination']
datBh[:, 5] = np.log10(Bham_data[ind]['Amplitude'])

print (datBh[:, 3])

ind = VGBs[name]['MM']
datM = np.zeros((len(MM_data[ind]), len(lbls)))
datM[:, 0] = MM_data[ind]['EclipticLongitude']
datM[:, 1] = MM_data[ind]['EclipticLatitude']
datM[:, 2] = MM_data[ind]['Frequency']
datM[:, 3] = np.log10(np.abs(MM_data[ind]['FrequencyDerivative']))
datM[:, 4] = MM_data[ind]['Inclination']
datM[:, 5] = np.log10(MM_data[ind]['Amplitude'])

ind = VGBs[name]['BC']
datBc = np.zeros((len(Barc_data[ind]), len(lbls)))
datBc[:, 0] = Barc_data[ind]['EclipticLongitude']
datBc[:, 1] = Barc_data[ind]['EclipticLatitude']
datBc[:, 2] = Barc_data[ind]['Frequency']
datBc[:, 3] = np.log10(np.abs(Barc_data[ind]['FrequencyDerivative']))
datBc[:, 4] = np.arccos(np.cos(Barc_data[ind]['Inclination']))
datBc[:, 5] = np.log10(Barc_data[ind]['Amplitude'])

ind = VGBs[name]['S']
datS = np.zeros((len(S_data[ind]), len(lbls)))
datS[:, 0] = S_data[ind]['EclipticLongitude']
datS[:, 1] = S_data[ind]['EclipticLatitude']
datS[:, 2] = S_data[ind]['Frequency']
datS[:, 3] = np.log10(np.abs(S_data[ind]['FrequencyDerivative']))
datS[:, 4] = np.arccos(np.cos(S_data[ind]['Inclination']))
datS[:, 5] = np.log10(S_data[ind]['Amplitude'])

bet, lam, Amp, f0, fdot, iota, psi, phi0 = GetParams(p, ind_0)

tr_s = np.array([lam, bet, f0, np.log10(fdot), iota, np.log10(Amp)])
print ("true params", tr_s)
print ('M', datM[:3, 0])
print ('Bh', datBh[:3,0])
print ('Bc', datBc[:3,0])
rng = [0.999, 0.999, 0.999, (-20.5,-15.5), 0.99, 0.99]
rng = []
# print (datBh.min(), datBc.min(), datM.min())
# print (max(datBh.max(), datBc.max(), datM.max()))
for i in range(len(lbls)):
    oner = ( min(datBh[:,i].min(), datS[:, i].min(), datM[:,i].min()), max(datBh[:, i].max(), datS[:,i].max(), datM[:,i].max()) )
    rng.append(oner)

fig2 =  corner.corner(datBh,  bins=40, hist_kwargs={'density':True, 'lw':3}, plot_datapoints=False, fill_contours=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='#348ABD', truths= tr_s, truth_color='k', use_math_test=True,\
                        levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datS,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color=plotpal[3], truths= tr_s, truth_color='k', use_math_test=True,\
                         levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datM,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, fill_contours=False,  labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='coral', truths= tr_s, truth_color='k', use_math_test=True,\
                         range=rng, levels=[0.9], title_kwargs={"fontsize": 12})

plt.show()
fig2.savefig("/home/stefan/Repositories/ldc1_evaluation/Stefan/VGB_"+str(ind_0)+".png")

In [ ]:
ind_0 = 5
name = 'GW322061'
inds_d = VGBs[name]

ind = VGBs[name]['BH']
datBh = np.zeros((len(Bham_data[ind]), len(lbls)))
datBh[:, 0] = Bham_data[ind]['EclipticLongitude'] -np.pi
datBh[:, 1] = Bham_data[ind]['EclipticLatitude']
datBh[:, 2] = Bham_data[ind]['Frequency']
datBh[:, 3] = np.log10(np.abs(Bham_data[ind]['FrequencyDerivative']))
datBh[:, 4] = Bham_data[ind]['Inclination']
datBh[:, 5] = np.log10(Bham_data[ind]['Amplitude'])

print (datBh[:, 3])

ind = VGBs[name]['MM']
datM = np.zeros((len(MM_data[ind]), len(lbls)))
datM[:, 0] = MM_data[ind]['EclipticLongitude']
datM[:, 1] = MM_data[ind]['EclipticLatitude']
datM[:, 2] = MM_data[ind]['Frequency']
datM[:, 3] = np.log10(np.abs(MM_data[ind]['FrequencyDerivative']))
datM[:, 4] = MM_data[ind]['Inclination']
datM[:, 5] = np.log10(MM_data[ind]['Amplitude'])

ind = VGBs[name]['BC']
datBc = np.zeros((len(Barc_data[ind]), len(lbls)))
datBc[:, 0] = Barc_data[ind]['EclipticLongitude']
datBc[:, 1] = Barc_data[ind]['EclipticLatitude']
datBc[:, 2] = Barc_data[ind]['Frequency']
datBc[:, 3] = np.log10(np.abs(Barc_data[ind]['FrequencyDerivative']))
datBc[:, 4] = np.arccos(np.cos(Barc_data[ind]['Inclination']))
datBc[:, 5] = np.log10(Barc_data[ind]['Amplitude'])

ind = VGBs[name]['S']
datS = np.zeros((len(S_data[ind]), len(lbls)))
datS[:, 0] = S_data[ind]['EclipticLongitude']
datS[:, 1] = S_data[ind]['EclipticLatitude']
datS[:, 2] = S_data[ind]['Frequency']
datS[:, 3] = np.log10(np.abs(S_data[ind]['FrequencyDerivative']))
datS[:, 4] = np.arccos(np.cos(S_data[ind]['Inclination']))
datS[:, 5] = np.log10(S_data[ind]['Amplitude'])


bet, lam, Amp, f0, fdot, iota, psi, phi0 = GetParams(p, ind_0)

tr_s = np.array([lam, bet, f0, np.log10(fdot), iota, np.log10(Amp)])
print ("true params", tr_s)
print ('M', datM[:3, 0])
print ('Bh', datBh[:3,0])
print ('Bc', datBc[:3,0])

rng = [0.999, 0.999, 0.999, (-20.5,-15.5), 0.99, 0.99]
rng = []
# print (datBh.min(), datBc.min(), datM.min())
# print (max(datBh.max(), datBc.max(), datM.max()))
for i in range(len(lbls)):
    oner = ( min(datBh[:,i].min(), datS[:, i].min(), datM[:,i].min()), max(datBh[:, i].max(), datS[:,i].max(), datM[:,i].max()) )
    rng.append(oner)

fig2 =  corner.corner(datBh,  bins=40, hist_kwargs={'density':True, 'lw':3}, plot_datapoints=False, fill_contours=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='#348ABD', truths= tr_s, truth_color='k', use_math_test=True,\
                        levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datS,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color=plotpal[3], truths= tr_s, truth_color='k', use_math_test=True,\
                         levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datM,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, fill_contours=False,  labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='coral', truths= tr_s, truth_color='k', use_math_test=True,\
                         range=rng, levels=[0.9], title_kwargs={"fontsize": 12})

plt.show()
fig2.savefig("/home/stefan/Repositories/ldc1_evaluation/Stefan/VGB_"+str(ind_0)+".png")

In [ ]:
ind_0 = 6
name = 'GW351250'
inds_d = VGBs[name]

ind = VGBs[name]['BH']
datBh = np.zeros((len(Bham_data[ind]), len(lbls)))
datBh[:, 0] = Bham_data[ind]['EclipticLongitude'] -np.pi
datBh[:, 1] = Bham_data[ind]['EclipticLatitude']
datBh[:, 2] = Bham_data[ind]['Frequency']
datBh[:, 3] = np.log10(np.abs(Bham_data[ind]['FrequencyDerivative']))
datBh[:, 4] = Bham_data[ind]['Inclination']
datBh[:, 5] = np.log10(Bham_data[ind]['Amplitude'])

print (datBh[:, 3])

ind = VGBs[name]['MM']
datM = np.zeros((len(MM_data[ind]), len(lbls)))
datM[:, 0] = MM_data[ind]['EclipticLongitude']-2.0*np.pi
datM[:, 1] = MM_data[ind]['EclipticLatitude']
datM[:, 2] = MM_data[ind]['Frequency']
datM[:, 3] = np.log10(np.abs(MM_data[ind]['FrequencyDerivative']))
datM[:, 4] = MM_data[ind]['Inclination']
datM[:, 5] = np.log10(MM_data[ind]['Amplitude'])

ind = VGBs[name]['BC']
datBc = np.zeros((len(Barc_data[ind]), len(lbls)))
datBc[:, 0] = Barc_data[ind]['EclipticLongitude']
datBc[:, 1] = Barc_data[ind]['EclipticLatitude']
datBc[:, 2] = Barc_data[ind]['Frequency']
datBc[:, 3] = np.log10(np.abs(Barc_data[ind]['FrequencyDerivative']))
datBc[:, 4] = np.arccos(np.cos(Barc_data[ind]['Inclination']))
datBc[:, 5] = np.log10(Barc_data[ind]['Amplitude'])

ind = VGBs[name]['S']
datS = np.zeros((len(S_data[ind]), len(lbls)))
datS[:, 0] = S_data[ind]['EclipticLongitude']
datS[:, 1] = S_data[ind]['EclipticLatitude']
datS[:, 2] = S_data[ind]['Frequency']
datS[:, 3] = np.log10(np.abs(S_data[ind]['FrequencyDerivative']))
datS[:, 4] = np.arccos(np.cos(S_data[ind]['Inclination']))
datS[:, 5] = np.log10(S_data[ind]['Amplitude'])


bet, lam, Amp, f0, fdot, iota, psi, phi0 = GetParams(p, ind_0)

tr_s = np.array([lam, bet, f0, np.log10(fdot), iota, np.log10(Amp)])
print ("true params", tr_s)
print ('M', datM[:3, 0])
print ('Bh', datBh[:3,0])
print ('Bc', datBc[:3,0])

rng = [0.999, 0.999, 0.999, (-20.5,-15.5), 0.99, 0.99]
rng = []
# print (datBh.min(), datBc.min(), datM.min())
# print (max(datBh.max(), datBc.max(), datM.max()))
for i in range(len(lbls)):
    oner = ( min(datBh[:,i].min(), datS[:, i].min(), datM[:,i].min()), max(datBh[:, i].max(), datS[:,i].max(), datM[:,i].max()) )
    rng.append(oner)

fig2 =  corner.corner(datBh,  bins=40, hist_kwargs={'density':True, 'lw':3}, plot_datapoints=False, fill_contours=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='#348ABD', truths= tr_s, truth_color='k', use_math_test=True,\
                        levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datS,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color=plotpal[3], truths= tr_s, truth_color='k', use_math_test=True,\
                         levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datM,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, fill_contours=False,  labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='coral', truths= tr_s, truth_color='k', use_math_test=True,\
                         range=rng, levels=[0.9], title_kwargs={"fontsize": 12})

plt.show()
fig2.savefig("/home/stefan/Repositories/ldc1_evaluation/Stefan/VGB_"+str(ind_0)+".png")

In [ ]:
ind_0 = 7
name = 'GW168350'
inds_d = VGBs[name]

ind = VGBs[name]['BH']
datBh = np.zeros((len(Bham_data[ind]), len(lbls)))
datBh[:, 0] = Bham_data[ind]['EclipticLongitude'] -np.pi
datBh[:, 1] = Bham_data[ind]['EclipticLatitude']
datBh[:, 2] = Bham_data[ind]['Frequency']
datBh[:, 3] = np.log10(np.abs(Bham_data[ind]['FrequencyDerivative']))
datBh[:, 4] = Bham_data[ind]['Inclination']
datBh[:, 5] = np.log10(Bham_data[ind]['Amplitude'])

print (datBh[:, 3])

ind = VGBs[name]['MM']
datM = np.zeros((len(MM_data[ind]), len(lbls)))
datM[:, 0] = MM_data[ind]['EclipticLongitude']
datM[:, 1] = MM_data[ind]['EclipticLatitude']
datM[:, 2] = MM_data[ind]['Frequency']
datM[:, 3] = np.log10(np.abs(MM_data[ind]['FrequencyDerivative']))
datM[:, 4] = MM_data[ind]['Inclination']
datM[:, 5] = np.log10(MM_data[ind]['Amplitude'])

ind = VGBs[name]['BC']
datBc = np.zeros((len(Barc_data[ind]), len(lbls)))
datBc[:, 0] = Barc_data[ind]['EclipticLongitude']
datBc[:, 1] = Barc_data[ind]['EclipticLatitude']
datBc[:, 2] = Barc_data[ind]['Frequency']
datBc[:, 3] = np.log10(np.abs(Barc_data[ind]['FrequencyDerivative']))
datBc[:, 4] = np.arccos(np.cos(Barc_data[ind]['Inclination']))
datBc[:, 5] = np.log10(Barc_data[ind]['Amplitude'])

ind = VGBs[name]['S']
datS = np.zeros((len(S_data[ind]), len(lbls)))
datS[:, 0] = S_data[ind]['EclipticLongitude']
datS[:, 1] = S_data[ind]['EclipticLatitude']
datS[:, 2] = S_data[ind]['Frequency']
datS[:, 3] = np.log10(np.abs(S_data[ind]['FrequencyDerivative']))
datS[:, 4] = np.arccos(np.cos(S_data[ind]['Inclination']))
datS[:, 5] = np.log10(S_data[ind]['Amplitude'])


bet, lam, Amp, f0, fdot, iota, psi, phi0 = GetParams(p, ind_0)

tr_s = np.array([lam, bet, f0, np.log10(fdot), iota, np.log10(Amp)])
print ("true params", tr_s)
print ('M', datM[:3, 0])
print ('Bh', datBh[:3,0])
print ('Bc', datBc[:3,0])

rng = [0.999, 0.999, 0.999, (-20.5,-15.5), 0.99, 0.99]
rng = []
# print (datBh.min(), datBc.min(), datM.min())
# print (max(datBh.max(), datBc.max(), datM.max()))
for i in range(len(lbls)):
    oner = ( min(datBh[:,i].min(), datS[:, i].min(), datM[:,i].min()), max(datBh[:, i].max(), datS[:,i].max(), datM[:,i].max()) )
    rng.append(oner)

fig2 =  corner.corner(datS,  bins=40, hist_kwargs={'density':True, 'lw':3}, plot_datapoints=False, fill_contours=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color=plotpal[3], truths= tr_s, truth_color='k', use_math_test=True,\
                        levels=[0.9], title_kwargs={"fontsize": 12})

# corner.corner(datS,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
#                         color=plotpal[3], truths= tr_s, truth_color='k', use_math_test=True,\
#                          levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datM,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, fill_contours=False,  labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='coral', truths= tr_s, truth_color='k', use_math_test=True,\
                          levels=[0.9], title_kwargs={"fontsize": 12})

plt.show()
fig2.savefig("/home/stefan/Repositories/ldc1_evaluation/Stefan/VGB_"+str(ind_0)+".png")

In [ ]:
ind_0 = 8
name = 'GW622028'
inds_d = VGBs[name]

ind = VGBs[name]['BH']
datBh = np.zeros((len(Bham_data[ind]), len(lbls)))
datBh[:, 0] = Bham_data[ind]['EclipticLongitude'] -np.pi
datBh[:, 1] = Bham_data[ind]['EclipticLatitude']
datBh[:, 2] = Bham_data[ind]['Frequency']
datBh[:, 3] = np.log10(np.abs(Bham_data[ind]['FrequencyDerivative']))
datBh[:, 4] = Bham_data[ind]['Inclination']
datBh[:, 5] = np.log10(Bham_data[ind]['Amplitude'])

print (datBh[:, 3])

ind = VGBs[name]['MM']
datM = np.zeros((len(MM_data[ind]), len(lbls)))
datM[:, 0] = MM_data[ind]['EclipticLongitude']
datM[:, 1] = MM_data[ind]['EclipticLatitude']
datM[:, 2] = MM_data[ind]['Frequency']
datM[:, 3] = np.log10(np.abs(MM_data[ind]['FrequencyDerivative']))
datM[:, 4] = MM_data[ind]['Inclination']
datM[:, 5] = np.log10(MM_data[ind]['Amplitude'])

ind = VGBs[name]['BC']
datBc = np.zeros((len(Barc_data[ind]), len(lbls)))
datBc[:, 0] = Barc_data[ind]['EclipticLongitude']
datBc[:, 1] = Barc_data[ind]['EclipticLatitude']
datBc[:, 2] = Barc_data[ind]['Frequency']
datBc[:, 3] = np.log10(np.abs(Barc_data[ind]['FrequencyDerivative']))
datBc[:, 4] = np.arccos(np.cos(Barc_data[ind]['Inclination']))
datBc[:, 5] = np.log10(Barc_data[ind]['Amplitude'])

ind = VGBs[name]['S']
datS = np.zeros((len(S_data[ind]), len(lbls)))
datS[:, 0] = S_data[ind]['EclipticLongitude']
datS[:, 1] = S_data[ind]['EclipticLatitude']
datS[:, 2] = S_data[ind]['Frequency']
datS[:, 3] = np.log10(np.abs(S_data[ind]['FrequencyDerivative']))
datS[:, 4] = np.arccos(np.cos(S_data[ind]['Inclination']))
datS[:, 5] = np.log10(S_data[ind]['Amplitude'])


bet, lam, Amp, f0, fdot, iota, psi, phi0 = GetParams(p, ind_0)

tr_s = np.array([lam, bet, f0, np.log10(fdot), iota, np.log10(Amp)])
print ("true params", tr_s)
print ('M', datM[:3, 0])
print ('Bh', datBh[:3,0])
print ('Bc', datBc[:3,0])

rng = [0.999, 0.999, 0.999, (-20.5,-15.5), 0.99, 0.99]
rng = []
# print (datBh.min(), datBc.min(), datM.min())
# print (max(datBh.max(), datBc.max(), datM.max()))
for i in range(len(lbls)):
    oner = ( min(datBh[:,i].min(), datS[:, i].min(), datM[:,i].min()), max(datBh[:, i].max(), datS[:,i].max(), datM[:,i].max()) )
    rng.append(oner)

fig2 =  corner.corner(datBh,  bins=40, hist_kwargs={'density':True, 'lw':3}, plot_datapoints=False, fill_contours=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='#348ABD', truths= tr_s, truth_color='k', use_math_test=True,\
                        levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datS,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color=plotpal[3], truths= tr_s, truth_color='k', use_math_test=True,\
                         levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datM,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, fill_contours=False,  labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='coral', truths= tr_s, truth_color='k', use_math_test=True,\
                          levels=[0.9], title_kwargs={"fontsize": 12})

plt.show()
fig2.savefig("/home/stefan/Repositories/ldc1_evaluation/Stefan/VGB_"+str(ind_0)+".png")

In [ ]:
ind_0 = 9
name = 'GW261301'
inds_d = VGBs[name]
s = 1
ind = VGBs[name]['BH']
datBh = np.zeros((len(Bham_data[ind]), len(lbls)))
datBh[:, 0] = Bham_data[ind]['EclipticLongitude'] -np.pi
datBh[:, 1] = Bham_data[ind]['EclipticLatitude']
datBh[:, 2] = Bham_data[ind]['Frequency']
datBh[:, 3] = np.log10(np.abs(Bham_data[ind]['FrequencyDerivative']))
datBh[:, 4] = Bham_data[ind]['Inclination']
datBh[:, 5] = np.log10(Bham_data[ind]['Amplitude'])

print (datBh[:, 3])

ind = VGBs[name]['MM']
datM = np.zeros((len(MM_data[ind]), len(lbls)))
datM[:, 0] = MM_data[ind]['EclipticLongitude']
datM[:, 1] = MM_data[ind]['EclipticLatitude']
datM[:, 2] = MM_data[ind]['Frequency']
datM[:, 3] = np.log10(np.abs(MM_data[ind]['FrequencyDerivative']))
datM[:, 4] = MM_data[ind]['Inclination']
datM[:, 5] = np.log10(MM_data[ind]['Amplitude'])

ind = VGBs[name]['BC']
datBc = np.zeros((len(Barc_data[ind]), len(lbls)))
datBc[:, 0] = Barc_data[ind]['EclipticLongitude']
datBc[:, 1] = Barc_data[ind]['EclipticLatitude']
datBc[:, 2] = Barc_data[ind]['Frequency']
datBc[:, 3] = np.log10(np.abs(Barc_data[ind]['FrequencyDerivative']))
datBc[:, 4] = np.arccos(np.cos(Barc_data[ind]['Inclination']))
datBc[:, 5] = np.log10(Barc_data[ind]['Amplitude'])

ind = VGBs[name]['S']
datS = np.zeros((len(S_data[ind]), len(lbls)))
datS[:, 0] = S_data[ind]['EclipticLongitude']
datS[:, 1] = S_data[ind]['EclipticLatitude']
datS[:, 2] = S_data[ind]['Frequency']
datS[:, 3] = np.log10(np.abs(S_data[ind]['FrequencyDerivative']))
datS[:, 4] = np.arccos(np.cos(S_data[ind]['Inclination']))
datS[:, 5] = np.log10(S_data[ind]['Amplitude'])


bet, lam, Amp, f0, fdot, iota, psi, phi0 = GetParams(p, ind_0)

tr_s = np.array([lam, bet, f0, np.log10(fdot), iota, np.log10(Amp)])
print ("true params", tr_s)
print ('M', datM[:3, 0])
print ('Bh', datBh[:3,0])
print ('Bc', datBc[:3,0])

rng = [0.999, 0.999, 0.999, (-20.5,-15.5), 0.99, 0.99]
rng = []
# print (datBh.min(), datBc.min(), datM.min())
# print (max(datBh.max(), datBc.max(), datM.max()))
for i in range(len(lbls)):
    oner = ( min(datBh[:,i].min(), datS[:, i].min(), datM[:,i].min()), max(datBh[:, i].max(), datS[:,i].max(), datM[:,i].max()) )
    rng.append(oner)

fig2 =  corner.corner(datBh,  bins=40, hist_kwargs={'density':True, 'lw':3}, plot_datapoints=False, fill_contours=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='#348ABD', truths= tr_s, truth_color='k', use_math_test=True,\
                        levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datS,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color=plotpal[3], truths= tr_s, truth_color='k', use_math_test=True,\
                         levels=[0.9], title_kwargs={"fontsize": 12})

corner.corner(datM,  bins=40,  hist_kwargs={'density':True, 'lw':3}, fig=fig2, plot_datapoints=False, fill_contours=False,  labels=lbls, quantiles=[0.05, 0.5, 0.95], show_titles=True, \
                        color='coral', truths= tr_s, truth_color='k', use_math_test=True,\
                         range=rng, levels=[0.9], title_kwargs={"fontsize": 12})

plt.show()
fig2.savefig("/home/stefan/Repositories/ldc1_evaluation/Stefan/VGB_"+str(ind_0)+".png")

#### Compute Jensen-Shannon divergence for the frequency

In [ ]:
for name in VGBs.keys():
    ind_d = VGBs[name]
    fr_Bh = Bham_data[ind_d['BH']]['Frequency']
    fr_MM = MM_data[ind_d['MM']]['Frequency']
    fr_Bc = (Barc_data[ind_d['BC']]['Frequency'])[::8]
    
    #computing pair-wise JSd 
    samplesBh_MM = [fr_Bh, fr_MM]
    JS_BhMM = jensen_shannon_divergence(samplesBh_MM)
    samplesBh_Bc =  [fr_Bh, fr_Bc]
    JS_BhBc = jensen_shannon_divergence(samplesBh_Bc)
    samplesMM_Bc =  [fr_MM, fr_Bc]
    JS_MMBc = jensen_shannon_divergence(samplesMM_Bc)
    
    print (name)
    print ('Bh-MM', JS_BhMM, 'Bh-Bc', JS_BhBc, 'MM-Bc', JS_MMBc)
    print (JS_BhMM, JS_BhBc,  JS_MMBc)
    

    
    
    

## Computing the likelihood with the submitted points

### Likelihood evaluation 



In [ ]:
fastGB = FastGB.FastGB("Test", dt=del_t, Tobs=Tobs, orbit="analytic")


def ComputeLogLikelihood(Subm):
    
    df = 1.0/Tobs
    bet = Subm['EclipticLatitude']
    lam = Subm['EclipticLongitude']
    Amp = Subm['Amplitude'] 
    f0 = Subm['Frequency']
    fdot = Subm['FrequencyDerivative']
    iota = Subm['Inclination']
    psi = Subm['Polarization']
    phi0 = Subm['InitialPhase']
    
    prm = np.array([f0, fdot, bet, lam, Amp, iota, psi, phi0]) 
    Xf, Yf, Zf = fastGB.onefourier(simulator='synthlisa', params=prm, buffer=None, T=Tobs, dt=del_t, algorithm='Michele', oversample=4)
    
        
#     Af = FrequencyArray.FrequencyArray( ((Zf[:] - Xf[:])/np.sqrt(2.0))/df, dtype=np.complex128, kmin=Xf.kmin, df=Xf.df)
#     Ef = FrequencyArray.FrequencyArray( ((Zf[:] - 2.0*Yf[:] + Xf[:])/np.sqrt(6.0))/df, dtype=np.complex128, kmin=Xf.kmin, df=Xf.df) 
    
    Af = (np.array(Zf[:] - Xf[:])/np.sqrt(2.0))/df
    Ef = ((Zf[:] - 2.0*Yf[:] + Xf[:])/np.sqrt(6.0))/df
    
     
    ib = Xf.kmin
    ie = Xf.kmin+len(Xf) 
    fr = np.arange(Xf.kmin, Xf.kmin+len(Xf))*df
    
    SA = tdi.noisepsd_AE(fr, model='Proposal', includewd=None)
    SNR2 = np.sum( np.real(Adf[ib:ie] * np.conjugate(Af.data) + Edf[ib:ie] * np.conjugate(Ef.data))/SA )
    hh = np.sum((np.absolute(Af.data)**2 + np.absolute(Ef.data)**2) /SA)
    
    loglik = 4.0*df*( SNR2 - 0.5 * hh )
    return (loglik)  
    
    
    
    
    
    

In [ ]:
i_upto = 10

for ii, name in enumerate(VGBs.keys()):
    ind_d = VGBs[name]
#     print(np.shape(Bham_data[ind_d['BH']]), np.shape(MM_data[ind_d['MM']]['Frequency'][::4]), np.shape(Barc_data[ind_d['BC']][::40]) )
    BHsampl = Bham_data[ind_d['BH']]
    MMsampl = MM_data[ind_d['MM']][::4]
    BCsampl = Barc_data[ind_d['BC']][::40]
    
    bet, lam, Amp, f0, fdot, iota, psi, phi0 = GetParams(p, ii)
    trs= {'EclipticLatitude': bet, 'EclipticLongitude': lam, 'Amplitude': Amp, 'Frequency': f0, \
          'FrequencyDerivative': fdot, 'Inclination': iota, 'Polarization': psi, 'InitialPhase': phi0}
    
    ### adjusting conventions:
    BHsampl['EclipticLongitude']  = BHsampl['EclipticLongitude'] -np.pi
    
    print (name, 'True', ComputeLogLikelihood(trs))
    for j in range(i_upto):
        s_BH = BHsampl[j]
        s_MM = MMsampl[j]
        s_BC = BCsampl[j]
#         print (s_BH['Frequency'], s_MM['Frequency'], s_BC['Frequency'])
        logBH = ComputeLogLikelihood(s_BH)
        logMM = ComputeLogLikelihood(s_MM)
        logBC = ComputeLogLikelihood(s_BC)
        
        print ('likelihoods, BH, MM, BC', logBH, logMM, logBC)
            
        



